# LIBRARIES & FUNCTIONS

In [1]:
# LIBRARIES #

import numpy as np

from scipy import optimize

from bokeh.plotting import figure, show
from bokeh.io import output_notebook
from bokeh.models import LinearColorMapper, BasicTicker, ColorBar, Plot, CustomJS, ColumnDataSource, Rect, Cross
from bokeh.layouts import row, gridplot, column
from bokeh.models.widgets import Slider, Button
from bokeh.events import ButtonClick

import SimpleITK as sitk 

import pydicom

from Gafchromic import GafchromicFilms

import time


output_notebook()

Loading BokehJS ...

In [2]:
# FONCTIONS #



# subsample data array : #
# @params:
#  array : array to subsample
#  sizex : size in x
#  sizey : size in y
#  subfactor :  subsampling factor
def subSampleDataArray(array, sizex, sizey, subfactor):
    newsizex = int(sizex/subfactor)
    newsizey = int(sizey/subfactor)
    
    newarray = array[0:newsizey*subfactor, 0:newsizex*subfactor]\
                        .reshape((newsizey, subfactor, newsizex, subfactor)).mean(3).mean(1)
    
    return newarray


# IMG READING & PROCESSING

In [3]:
# INPUT PARAMETERS:
# <!> ne pas mettre d'accent dans les chemins et noms de fichiers



# Variables:
m_path = 'testRBGB/'
m_nbOfFiles = 1
m_firstNb = 1
# m_GafFilesName = "testRBGB_254pp_bord_00"
m_GafFilesName = "testRBGB_00"
m_fileExtension = ".tif"
m_splineFile = 'G:/Commun/PHYSICIENS/Erwann/EBT3/13 - etalonnage lot 02282001/scan 24h/bSpline_data.txt'

m_doseFileName = "RD.dcm"
m_planFileName =  "RP.dcm"

m_dimViewer = 600




In [4]:
# READS THE FILES AND DISPLAY PROFILES :

ds_dose = pydicom.read_file(m_path+m_doseFileName)
ds_plan = pydicom.read_file(m_path+m_planFileName)


print('Reading dose files:')

# Isocenter Coordinates from plan file:
coord_iso_x=ds_plan.BeamSequence[0].ControlPointSequence[0].IsocenterPosition[0]
coord_iso_y=ds_plan.BeamSequence[0].ControlPointSequence[0].IsocenterPosition[1]
coord_iso_z=ds_plan.BeamSequence[0].ControlPointSequence[0].IsocenterPosition[2]

print ('  > isocenter coordinates = (', coord_iso_x, ";", coord_iso_y, ";", coord_iso_z, ")")


# Dose image coordinates from dose file:
position_image=list(ds_dose['0020','0032'].value) # +x left, +y post, +z head   
position_image_x=list(ds_dose['0020','0032'].value)[0]
position_image_y=list(ds_dose['0020','0032'].value)[1]
position_image_z=list(ds_dose['0020','0032'].value)[2]

print ('  > Dose image position = (', position_image_x, ";", position_image_y, ";", position_image_z, ")")


# Isocenter: 
Z_centre = int(ds_plan.BeamSequence[0].ControlPointSequence[0].IsocenterPosition[2]-int(position_image_z))
iso_X =int(ds_plan.BeamSequence[0].ControlPointSequence[0].IsocenterPosition[0]-int(position_image_x))
iso_Y =int(ds_plan.BeamSequence[0].ControlPointSequence[0].IsocenterPosition[1]-int(position_image_y))

print ('  > isocenter position = (', iso_X, ";", iso_Y, ";", Z_centre, ")")


# Dose matrix in axial plane at isocenter:
dim_dose= ds_dose.pixel_array.shape
print ('  > dose matrix size = ', dim_dose)

px=pixel_spacing_x=(ds_dose['0028','0030'].value[0]) #definition de la résolution de la matrice de dose
pixel_spacing_y=(ds_dose['0028','0030'].value[1])
slice_spacing_z=(ds_dose['3004','000C'].value[1])
print ('  > pixel spacing = (', px, ";", pixel_spacing_y, ";", slice_spacing_z, ")")

coord_iso_dose = (round(iso_X/pixel_spacing_x),round(iso_Y/pixel_spacing_y),round(Z_centre/slice_spacing_z))
print ('  > isocenter coodinates in dose matrix =', coord_iso_dose)

doseimg = ds_dose.pixel_array[:,coord_iso_dose[1],:] * ds_dose.DoseGridScaling * 100 #en cGy

print ('  > maximum dose :', np.amax(doseimg))


p = figure(plot_width=m_dimViewer, plot_height=int(m_dimViewer*dim_dose[0]/dim_dose[2]), 
           title='dose img', toolbar_location="above")
p.image(image=[doseimg], x=0, y=0, dw=dim_dose[2], dh=dim_dose[0], palette="Plasma256")
show(p)

Reading dose files:
  > isocenter coordinates = ( 0.00 ; -100.99 ; 0.00 )
  > Dose image position = ( -201.986715 ; -202.360287 ; -200.000000 )
  > isocenter position = ( 201 ; 101 ; 200 )
  > dose matrix size =  (401, 202, 203)
  > pixel spacing = ( 2.000000 ; 2.000000 ; 1.00 )
  > isocenter coodinates in dose matrix = (100, 50, 200)
  > maximum dose : 743.7617611200001


In [5]:
# Tiff to Dose conversion:

try:
    g = GafchromicFilms(m_path+m_GafFilesName, m_firstNb, m_nbOfFiles)
    gafdoseimg = g.convertToDose_cubicSplineFit(m_splineFile, 800)
    size = g.getSize()
except ValueError as err:
    print('Erreur: ' + err)
    

newgafdoseimg = subSampleDataArray(gafdoseimg, size[0], size[1], 10)
# print(newdoseimg.shape)


p = figure(plot_width=m_dimViewer, plot_height=int(m_dimViewer*size[1]/size[0]), 
           title='Gafchromicdose img', toolbar_location="above")
# p.image(image=[gafdoseimg], x=0, y=0, dw=size[0], dh=size[1], palette="Plasma256")
p.image(image=[newgafdoseimg], x=0, y=0, dw=newgafdoseimg.shape[1], dh=newgafdoseimg.shape[0], palette="Plasma256")

show(p)

In [16]:
# Plots profiles:


offset = -26

p = figure(plot_width=600, plot_height=400, title="Dose calculated vs real", toolbar_location="above")

x1 = np.arange(0, len(doseimg[:,100]), 1)
x2 = np.arange(0, len(newgafdoseimg[:,150]), 1)

p.line(x1+offset, np.flip(doseimg[:,100]), line_width=2, line_color='firebrick')
p.line(x2, newgafdoseimg[:,150], line_width=2, line_color='black', line_dash='dotted')

show(p)

401
419


In [ ]:
# INPUT PARAMETERS:
# <!> ne pas mettre d'accent dans les chemins et noms de fichiers


m_path = 'testRBGB/'
m_nbOfFiles = 1
m_firstNb = 1
# m_filesName = "testRBGB_254pppnatif_00"
# m_filesName = "testRBGB_1270vers254ppp_00"
m_filesName = "testRBGB_254pp_bord_00"
m_fileExtension = ".tif"

m_dimViewer = 600


# Rectangles on the left side
leftRectList = []
leftRectList.append([1300,600,1350,2900])


# Rectangles on the right side
rightRectList = []
rightRectList.append([2900,600,2950,2900])


# Loads the image:
img = sitk.ReadImage(m_path+m_filesName+str(m_firstNb)+m_fileExtension)

sizex = img.GetWidth()
sizey = img.GetHeight()
imgOrigin = img.GetOrigin()
imgSpacing = img.GetSpacing()

size = (sitk.GetArrayFromImage(img).shape[0], 
        sitk.GetArrayFromImage(img).shape[1], 
        sitk.GetArrayFromImage(img).shape[2], 
        m_nbOfFiles)
imgs = np.zeros(size)

for i in range(m_nbOfFiles):
    img = sitk.ReadImage(m_path+m_filesName+str(m_firstNb+i)+m_fileExtension)
    imgs[:,:,:,i] = sitk.GetArrayFromImage(img)
    array = np.median(imgs, axis=3)


# Display
#displayRectanglesOnImage(m_path+m_filesName+str(m_firstNb)+m_fileExtension, leftRectList, m_dimViewer)

print("dONE !")

In [ ]:
subsampleFactor = 10

newarray = subSampleDataArray(array, sizex, sizey, subsampleFactor)

# Rectangles de l'image correspondant aux doses énoncées au dessus
# subrectList = []
# for i in range(len(rectList)):
#     subrectList.append([int(rectList[i][0]/subsampleFactor), 
#                        int(rectList[i][1]/subsampleFactor),
#                        int(rectList[i][2]/subsampleFactor),
#                        int(rectList[i][3]/subsampleFactor)])

subrectList1 = []
for i in range(len(rightRectList)):
    subrectList1.append([int(rightRectList[i][0]/subsampleFactor), 
                       int(rightRectList[i][1]/subsampleFactor),
                       int(rightRectList[i][2]/subsampleFactor),
                       int(rightRectList[i][3]/subsampleFactor)])

subrectList2 = []
for i in range(len(leftRectList)):
    subrectList2.append([int(leftRectList[i][0]/subsampleFactor), 
                       int(leftRectList[i][1]/subsampleFactor),
                       int(leftRectList[i][2]/subsampleFactor),
                       int(leftRectList[i][3]/subsampleFactor)])

    
# visu2D(newarray[:,:,0]/newarray[:,:,2], newarray[:,:,1]/newarray[:,:,2], subrectList1, m_dose,
#       title='G over B values for different doses',
#       titleX='G values',
#       titleY='B values',
#       alpha = 0.1)


arrayr = newarray[:,:,0]
arrayg = newarray[:,:,1]
arrayb = newarray[:,:,2]

visu2D(arrayr/arrayb, arrayg/arrayb, subrectList1, m_dose,
      title='G over B values for different doses',
      titleX='R/B values',
      titleY='G/B values',
      alpha = 1)

# visu2Dcompare(arrayr/arrayb, arrayg/arrayb, subrectList2, subrectList1,
#       title='G over B values for different doses',
#       titleX='R/B values',
#       titleY='G/B values',
#       alpha = 0.1)

# ETALONNAGE R/B & G/B

In [ ]:
# ETALONNAGE NUAGES DE POINTS #

# To be set:
rectlist = rightRectList
subsampleFactor = 10

alpha = 0.2
color1 = 'darkblue'
color2 = 'firebrick'
signSize = 3





# Fitting function:
# def func(x, a, b, c, d, e):
#     return a*x[:,0]*x[:,0] + b*x[:,0] + c*x[:,1]*x[:,1] + d*x[:,1] + e
def func(x, a, b, c, d):
    return np.exp(a*x[:,0] + b*x[:,1] + c) + d





# Resampling of data array and rectangles:
newarray = subSampleDataArray(array, sizex, sizey, subsampleFactor)

subrectlist = []
for i in range(len(rectlist)):
    subrectlist.append([int(rectlist[i][0]/subsampleFactor), 
                       int(rectlist[i][1]/subsampleFactor),
                       int(rectlist[i][2]/subsampleFactor),
                       int(rectlist[i][3]/subsampleFactor)])


# Counts nb of pixels in all rectangles:
totalNbPix = 0
for i in range(len(subrectlist)):
    totalNbPix += (subrectlist[i][2]-subrectlist[i][0])*(subrectlist[i][3]-subrectlist[i][1])


# Fill in the X, Y and dose tables
doseArr = np.zeros(totalNbPix)
rbAndGbArr = np.zeros((totalNbPix,2))

k=0
meanRGB = np.zeros((len(subrectlist),3))
for i in range(len(subrectlist)):
    arrRtmp = newarray[subrectlist[i][1]:subrectlist[i][3],subrectlist[i][0]:subrectlist[i][2],0].flatten()
    arrGtmp = newarray[subrectlist[i][1]:subrectlist[i][3],subrectlist[i][0]:subrectlist[i][2],1].flatten()
    arrBtmp = newarray[subrectlist[i][1]:subrectlist[i][3],subrectlist[i][0]:subrectlist[i][2],2].flatten()

    meanRGB[i,0] = np.mean(arrRtmp)
    meanRGB[i,1] = np.mean(arrGtmp)
    meanRGB[i,2] = np.mean(arrBtmp)

    for j in range(len(arrRtmp)):
        doseArr[k] = m_dose[i]
        rbAndGbArr[k,0] = arrRtmp[j] / arrBtmp[j]
        rbAndGbArr[k,1] = arrGtmp[j] / arrBtmp[j]
        k += 1
        


# fitting:
params, params_covariance = optimize.curve_fit(func, rbAndGbArr, doseArr)

print('Parameters:')
print('   - a:', params[0])
print('   - b:', params[1])
print('   - c:', params[2])
print('   - d:', params[3])
# print('   - e:', params[4])
        


# Display
p1 = figure(plot_width=600, plot_height=400, title="R/B and G/B over Dose", toolbar_location="above")
p1.circle(rbAndGbArr[:,0], doseArr, size=signSize, color=color1, alpha=alpha, legend_label="R/B")
p1.circle(rbAndGbArr[:,1], doseArr, size=signSize, color=color2, alpha=alpha, legend_label="G/B")

src = ColumnDataSource(dict(x=meanRGB[:,0]/meanRGB[:,2], y=m_dose))
glyph = Cross(x="x", y="y", size=10, line_color='green', fill_color=None, line_width=2)
p1.add_glyph(src, glyph)


p2 = figure(plot_width=600, plot_height=400, title="Dose calculated vs real", toolbar_location="above")
x = np.zeros((len(m_dose),2))
x[:,0] = meanRGB[:,0]/meanRGB[:,2]
x[:,1] = meanRGB[:,1]/meanRGB[:,2]
p2.line(m_dose, func(x, params[0], params[1], params[2], params[3]), line_width=2, line_color='firebrick')
# p2.line(m_dose, func(x, params[0], params[1], params[2], params[3], params[4]), line_width=2, line_color='firebrick')
p2.line(m_dose,m_dose, line_width=1, line_color='black', line_dash='dotted')


p3 = figure(plot_width=600, plot_height=400, title="Dose calculation Error in %", toolbar_location="above")
p3.line(m_dose,(m_dose-func(x, params[0], params[1], params[2], params[3]))/m_dose*100, line_width=2, line_color='firebrick')
# p3.line(m_dose,(m_dose-func(x, params[0], params[1], params[2], params[3], params[4]))/m_dose*100, line_width=2, line_color='firebrick')
p3.line(m_dose,2 , line_width=1, line_color='black', line_dash='dotted')
p3.line(m_dose,-2 , line_width=1, line_color='black', line_dash='dotted')


grid = gridplot([[p1],[p2],[p3]])
show(grid)
    
print("  >>> dONE !")

In [ ]:
# ETALONNAGE VALEURS MOYENNES #

# To be set:
rectlist = rightRectList
subsampleFactor = 10

alpha = 1.0
color1 = 'darkblue'
color2 = 'firebrick'
signSize = 3





# Fitting function:
def func(x, a, b, c, d, e, f, g):
    return a*(x[:,0]*x[:,1])+  b*(x[:,0]**d*x[:,1]**c)

#    return np.exp(a*x[:,0] + b*x[:,1] + c) + d
#    return a*np.exp(b*x[:,0] + c) + d*np.exp(e*x[:,1] + f) + g   # pas mieux que sans a et d
#    return np.exp(a*x[:,0] + b) + np.exp(c*x[:,1] + d) + e
#    return a*x[:,0]*x[:,0] + b*x[:,0] + c*x[:,1]*x[:,1] + d*x[:,1] + e   # pas top
#    return a*x[:,0]*x[:,0]*x[:,0] + b*x[:,0]*x[:,0] + c*x[:,0] + d 
#    return a*x[:,0]**b + c*x[:,1]**d + e 


# Resampling of data array and rectangles:
newarray = subSampleDataArray(array, sizex, sizey, subsampleFactor)

subrectlist = []
for i in range(len(rectlist)):
    subrectlist.append([int(rectlist[i][0]/subsampleFactor), 
                       int(rectlist[i][1]/subsampleFactor),
                       int(rectlist[i][2]/subsampleFactor),
                       int(rectlist[i][3]/subsampleFactor)])


# fill in the arrays:
rbAndGbArr = np.zeros((len(subrectlist),2))
meanRGB = np.zeros((len(subrectlist),3))


for i in range(len(subrectlist)):
    arrRtmp = newarray[subrectlist[i][1]:subrectlist[i][3],subrectlist[i][0]:subrectlist[i][2],0].flatten()
    arrGtmp = newarray[subrectlist[i][1]:subrectlist[i][3],subrectlist[i][0]:subrectlist[i][2],1].flatten()
    arrBtmp = newarray[subrectlist[i][1]:subrectlist[i][3],subrectlist[i][0]:subrectlist[i][2],2].flatten()

    meanRGB[i,0] = np.mean(arrRtmp)
    meanRGB[i,1] = np.mean(arrGtmp)
    meanRGB[i,2] = np.mean(arrBtmp)

    rbAndGbArr[i, 0] = meanRGB[i,0] / meanRGB[i,2]
    rbAndGbArr[i, 1] = meanRGB[i,1] / meanRGB[i,2]


# fitting:
params, params_covariance = optimize.curve_fit(func, rbAndGbArr, m_dose)

print('Parameters:')
print('   - a:', params[0])
print('   - b:', params[1])
print('   - c:', params[2])
print('   - d:', params[3])
print('   - e:', params[4])
print('   - f:', params[5])
print('   - g:', params[5])
       


# Display
p1 = figure(plot_width=600, plot_height=400, title="R/B and G/B over Dose", toolbar_location="above")
p1.circle(rbAndGbArr[:,0], m_dose, size=signSize, color=color1, alpha=alpha, legend_label="R/B")
p1.circle(rbAndGbArr[:,1], m_dose, size=signSize, color=color2, alpha=alpha, legend_label="G/B")

p2 = figure(plot_width=600, plot_height=400, title="Dose calculated vs real", toolbar_location="above")
x = np.zeros((len(m_dose),2))
x[:,0] = meanRGB[:,0]/meanRGB[:,2]
x[:,1] = meanRGB[:,1]/meanRGB[:,2]
p2.line(m_dose, func(x, params[0], params[1], params[2], params[3], params[4], params[5], params[6]), line_width=2, line_color='firebrick')
p2.line(m_dose,m_dose, line_width=1, line_color='black', line_dash='dotted')


p3 = figure(plot_width=600, plot_height=400, title="Dose calculation Error in %", toolbar_location="above")
p3.line(m_dose,(m_dose-func(x, params[0], params[1], params[2], params[3], params[4], params[5], params[6]))/m_dose*100, line_width=2, line_color='firebrick')
p3.line(m_dose,1 , line_width=1, line_color='black', line_dash='dotted')
p3.line(m_dose,-1 , line_width=1, line_color='black', line_dash='dotted')


grid = gridplot([[p1],[p2],[p3]])
show(grid)
    
print("  >>> dONE !")

In [ ]:
# RAJOUT DE R/G #

# To be set:
rectlist = leftRectList
subsampleFactor = 10

alpha = 0.2
color1 = 'darkblue'
color2 = 'firebrick'
color3 = 'darkseagreen'
signSize = 3





# Fitting function:
# def func(x, a, b, c, d, e):
#     return a*x[:,0]*x[:,0] + b*x[:,0] + c*x[:,1]*x[:,1] + d*x[:,1] + e
def func(x, a, b, c, d, e):
    return np.exp(a*x[:,0] + b*x[:,1] + c*x[:,2] + d) + e





# Resampling of data array and rectangles:
newarray = subSampleDataArray(array, sizex, sizey, subsampleFactor)

subrectlist = []
for i in range(len(rectlist)):
    subrectlist.append([int(rectlist[i][0]/subsampleFactor), 
                       int(rectlist[i][1]/subsampleFactor),
                       int(rectlist[i][2]/subsampleFactor),
                       int(rectlist[i][3]/subsampleFactor)])


# Counts nb of pixels in all rectangles:
totalNbPix = 0
for i in range(len(subrectlist)):
    totalNbPix += (subrectlist[i][2]-subrectlist[i][0])*(subrectlist[i][3]-subrectlist[i][1])


# Fill in the X, Y and dose tables
doseArr = np.zeros(totalNbPix)
rbAndGbArr = np.zeros((totalNbPix,3))

k=0
meanRGB = np.zeros((len(subrectlist),3))
for i in range(len(subrectlist)):
    arrRtmp = newarray[subrectlist[i][1]:subrectlist[i][3],subrectlist[i][0]:subrectlist[i][2],0].flatten()
    arrGtmp = newarray[subrectlist[i][1]:subrectlist[i][3],subrectlist[i][0]:subrectlist[i][2],1].flatten()
    arrBtmp = newarray[subrectlist[i][1]:subrectlist[i][3],subrectlist[i][0]:subrectlist[i][2],2].flatten()

    meanRGB[i,0] = np.mean(arrRtmp)
    meanRGB[i,1] = np.mean(arrGtmp)
    meanRGB[i,2] = np.mean(arrBtmp)

    for j in range(len(arrRtmp)):
        doseArr[k] = m_dose[i]
        rbAndGbArr[k,0] = arrRtmp[j] / arrBtmp[j]
        rbAndGbArr[k,1] = arrGtmp[j] / arrBtmp[j]
        rbAndGbArr[k,2] = arrRtmp[j] / arrGtmp[j]
        k += 1
        


# fitting:
params, params_covariance = optimize.curve_fit(func, rbAndGbArr[1:, :], doseArr[1:])

print('Parameters:')
print('   - a:', params[0])
print('   - b:', params[1])
print('   - c:', params[2])
print('   - d:', params[3])
print('   - e:', params[4])
        


# Display
p1 = figure(plot_width=600, plot_height=400, title="R/B and G/B over Dose", toolbar_location="above")
p1.circle(rbAndGbArr[:,0], doseArr, size=signSize, color=color1, alpha=alpha, legend_label="R/B")
p1.circle(rbAndGbArr[:,1], doseArr, size=signSize, color=color2, alpha=alpha, legend_label="G/B")
p1.circle(rbAndGbArr[:,2], doseArr, size=signSize, color=color3, alpha=alpha, legend_label="R/G")

src = ColumnDataSource(dict(x=meanRGB[:,0]/meanRGB[:,2], y=m_dose))
glyph = Cross(x="x", y="y", size=10, line_color='green', fill_color=None, line_width=2)
p1.add_glyph(src, glyph)


p2 = figure(plot_width=600, plot_height=400, title="Dose calculated vs real", toolbar_location="above")
x = np.zeros((len(m_dose),3))
x[:,0] = meanRGB[:,0]/meanRGB[:,2]
x[:,1] = meanRGB[:,1]/meanRGB[:,2]
x[:,2] = meanRGB[:,0]/meanRGB[:,1]
p2.line(m_dose, func(x, params[0], params[1], params[2], params[3], params[4]), line_width=2, line_color='firebrick')
p2.line(m_dose,m_dose, line_width=1, line_color='black', line_dash='dotted')


p3 = figure(plot_width=600, plot_height=400, title="Dose calculation Error in %", toolbar_location="above")
p3.line(m_dose,(m_dose-func(x, params[0], params[1], params[2], params[3], params[4]))/m_dose*100, line_width=2, line_color='firebrick')
p3.line(m_dose,2 , line_width=1, line_color='black', line_dash='dotted')
p3.line(m_dose,-2 , line_width=1, line_color='black', line_dash='dotted')


grid = gridplot([[p1],[p2],[p3]])
show(grid)
    
print("  >>> dONE !")

In [ ]:
toDoseImg = subSampleDataArray(array[600:3800, 500:2500, :], 2000, 3200, 10)


arrayRB = toDoseImg[:,:,0] / toDoseImg[:,:,2]
arrayGB = toDoseImg[:,:,1] / toDoseImg[:,:,2]

doseimg = params[0]*arrayRB**params[1] + params[2]*arrayGB**params[3] + params[4]

simpleImageDisplay(doseimg, doseimg.shape[1], doseimg.shape[0], 80)


# DISPLAY HISTOS ETC...

In [ ]:
img = array[700:1400,600:2600,:]

basicImgDisplay(img, dimviewer=850)

plotHisto(img)

# minv = np.amin(img[:,:,0])
# basicBWImgDisplay(img[:,:,0], dimviewer=800, minvalue = minv, maxv = 60000, title='Red channel')

# minv = np.amin(img[:,:,1])
# basicBWImgDisplay(img[:,:,1], dimviewer=800, minvalue = minv, maxv = 60000, title='Green channel')

# minv = np.amin(img[:,:,2])
# basicBWImgDisplay(img[:,:,2], dimviewer=800, minvalue = minv, maxv = 60000, title='Blue channel')

# VISU 3D TESTS

In [ ]:
from mpl_toolkits.mplot3d import Axes3D

import matplotlib.pyplot as plt
import numpy as np

#%matplotlib inline
%matplotlib widget





fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')

for i in range(len(rectList)):
    rectValues = array[rectList[i][1]:rectList[i][3], rectList[i][0]:rectList[i][2], :]
    ax.scatter(rectValues[:,:,0].flatten(), rectValues[:,:,1].flatten(), 
               rectValues[:,:,2].flatten(), marker='o', color='darkblue')

ax.set_xlabel('X Label')
ax.set_ylabel('Y Label')
ax.set_zlabel('Z Label')

plt.show()